In [21]:
import json
import os
import math
import librosa
import numpy as np
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

DATASET_PATH = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original"
JSON_PATH = "/kaggle/working/All_Data.json"
Model_PATH = "/kaggle/working/Model.keras"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

**Preparing Data**

In [2]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.

        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:
                try:
		# load audio file
                    file_path = os.path.join(dirpath, f)
                    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                    for d in range(num_segments):

                    # calculate start and finish sample for current segment
                        start = samples_per_segment * d
                        finish = start + samples_per_segment

                    # extract mfcc
                        mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                        mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print("{}, segment:{}".format(file_path, d+1))
                except:
                      print("An exception occurred")

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [3]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


Processing: disco
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:1
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:2
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:3
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:4
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:5
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:6
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:7
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:8
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav, segment:9


/tmp/ipykernel_47/3254178244.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


An exception occurred
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:1
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:2
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:3
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:4
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:5
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:6
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:7
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:8
/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00041.wav, segment:9
/kaggle/input/g

In [13]:
def load_data(dataset_path):
    with open(dataset_path,"r") as fp:
        data = json.load(fp)
    x = np.array(data["mfcc"])
    y = np.array(data["labels"])
    print("Data succesfully loaded!")
    return x,y

In [23]:
def plot_history(history):
    fig,axs = plt.subplots(2)
    axs[0].plot(history.history["accuracy"],label="train accuracy")
    axs[0].plot(history.history["val_accuracy"],label="test accuracy")
    axs[0].set_ylabel("accuracy")
    axs[0].legend(loc="lower right")
    axs.set_title("Accuracy eval")
    
    axs[1].plot(history.history["loss"],label="train error")
    axs[1].plot(history.history["val_loss"],label="test error")
    axs[1].set_ylabel("error")    
    axs[1].set_xlabel("epoch")
    axs[1].legend(loc="upper right")
    axs.set_title("Error eval")
    
    plt.show()


In [24]:
fig,axs = plt.subplots(2)
x,y= load_data(JSON_PATH)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
model = keras.Sequential([
        keras.layers.Flatten(input_shape=(x.shape[1],x.shape[2])),
        keras.layers.Dense(512,activation="relu",kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256,activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64,activation="relu",kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10,activation="softmax")
        ])
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()
history = model.fit(x_train,y_train,validation_data = ( x_test,y_test),batch_size = 32,epochs = 300)
model.save(Model_PATH)
plot_history(history)

Exception ignored in: <function _xla_gc_callback at 0x7a2fd9cf0310>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7a2fd9cf0310>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Data succesfully loaded!
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 1690)              0         
                                                                 
 dense_20 (Dense)            (None, 512)               865792    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 256)               131328    
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 64)                16448     
                                                                 
 dropout_17 (Dropout)        


KeyboardInterrupt


KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x7a305c8a2560> (for post_execute):
Error in callback <function flush_figures at 0x7a305c8a1870> (for post_execute):



KeyboardInterrupt

